In [1]:
import pandas as pd
import seaborn as sns
import re
import matplotlib

,name,price,params,desc,region
2744141438,"EXEED TXL 2.0 AMT, 2023",4680000,"2.0 AMT (197 л.с.), внедорожник, полный, бензин",Покупайте Exeed с выгодой до 1 150 000 ₽ в лиз...,Санкт-Петербург
3464828016,"EXEED RX 2.0 AMT, 2023",5450000,"2 AMT (249 л.с.), внедорожник, полный, бензин",Сильвер официальный дилер Exeed предлагает уни...,Челябинск
3493473950,"EXEED TXL 2.0 AMT, 2023",4680000,"2.0 AMT (197 л.с.), внедорожник, полный, бензин",Выгода На Exeed Txl ДО 1 590 000 Рублей! Офици...,Краснодар
3467133132,"EXEED TXL 1.6 AMT, 2023",4000000,"1.6 AMT (186 л.с.), внедорожник, полный, бензин",Влюбитесь в каждое мгновенье с новым автомобил...,Санкт-Петербург
3278163543,"EXEED TXL 1.6 AMT, 2023",4260000,"1.6 AMT (186 л.с.), внедорожник, полный, бензин","Exeed — бренд премиальных автомобилей, созданн...",Ростов-на-Дону


In [2]:
def prepare_data(path):

    df = pd.read_csv(path, index_col=0)
    df[['mileage', 'engine_capacity', 'transmission', 'engine_power', 'body_type', 'wheel', 'fuel_type', 'model', 'model_2', 'year']] = None

    for i, row in df.loc[:, ['name', 'params', 'region']].iterrows():

        try:
            #Предобработка params
            mileage = re.search(r'([\d ]+) км,', row['params'], re.IGNORECASE)
            if mileage == None:
                mileage = 0
            else:
                df.loc[i, 'mileage'] = mileage.group(1).replace(' ', '')

            engine_capacity = re.search(r'([\d\.]+) [A-Z]+', row['params'])

            if engine_capacity != None:
                df.loc[i, 'engine_capacity'] = engine_capacity.group(1)

            df.loc[i, 'transmission'] = re.search(r'MT|CVT|AMT|AT', row['params'])[0]
            df.loc[i, 'engine_power'] = re.search(r'\((\d+) л\.с\.\),', row['params']).group(1)


            df.loc[i, 'body_type'] = re.search(r'([\w -]+), (полный|задний|передний)', row['params'], re.IGNORECASE).group(1)
            df.loc[i, 'wheel'] = re.search(r'полный|задний|передний', row['params'], re.IGNORECASE)[0]
            df.loc[i, 'fuel_type'] = re.search(r'бензин|дизель|электро|гибрид|газ', row['params'], re.IGNORECASE)[0]


            #предобработка name
            df.loc[i, 'model'] = re.search(r'[\w-]+ ', row['name'], re.IGNORECASE)[0]
            df.loc[i, 'model_2'] = re.search(r'[\w-]+ ([^,]*?) [\d\. ]*(AT|MT|AMT|CVT),', row['name'], re.IGNORECASE).group(1)        
            df.loc[i, 'year'] = re.search(r', ?(\d{4})', row['name']).group(1)

            #предобработка region
            df.loc[i, 'region'] = re.search(r'([^,]+),?', row['region'], re.IGNORECASE).group(1)

        except Exception as ex:
            print(row)
            print(ex)

    return df

In [4]:
df = pd.DataFrame()
for i in range(28):
    if i == 0:
        res = prepare_data(f'./datasets/data.csv')
    else:
        res = prepare_data(f'./datasets/data{i}.csv')
    
    df = pd.concat([df, res])


name                                          Audi A6, 2011
params    186 000 км, 2.0 CVT (180 л.с.), седан, передни...
region                              Иркутская обл., Иркутск
Name: 2978798022, dtype: object
'NoneType' object has no attribute 'group'
name                                           BMW X5, 2008
params    300 000 км, 3.0 AT (272 л.с.), внедорожник, по...
region                                                Минск
Name: 2634543434, dtype: object
'NoneType' object has no attribute 'group'
name         Changan Lamore 1.5 AMT, 2023
params    1.5 AMT седан, передний, бензин
region               поселение Московский
Name: 3746407298, dtype: object
'NoneType' object has no attribute 'group'
name                                Chevrolet Lacetti, 2010
params    280 000 км, 1.6 MT (109 л.с.), хетчбэк, передн...
region    Курская обл., Мантуровский р-н, 2-й Засеймский...
Name: 2476088237, dtype: object
'NoneType' object has no attribute 'group'
name                              

In [5]:
df.to_csv('./dataset.csv')